In [0]:
%tensorflow_version 2.x

In [0]:
import pathlib
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

In [0]:
x = [-1, 0, 1, 2, 3, 4]
y = [-3, -1, 1, 3, 5, 7]

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(1, input_shape=[1])])
model.compile(optimizer='sgd',
              loss='mse')
model.fit(x, y, epochs=200)

In [0]:
# Gereate saved model
export_dir = 'saved_model/1'
tf.saved_model.save(model, export_dir)

In [0]:
# Convert the SavedModel to TFLite
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()

In [0]:
type(tflite_model)

In [0]:
tflite_model_file = pathlib.Path('model.tflite')
tflite_model_file.write_bytes(tflite_model)

In [0]:
# Initialize the TFLite Interpreter to try it out
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [0]:
# Test the TensorFlow Lite model on random input data.
input_shape = input_details[0]['shape']
inputs, outputs = [], []
for _ in range(100):
  input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
  interpreter.set_tensor(input_details[0]['index'], input_data)

  interpreter.invoke()
  tflite_results = interpreter.get_tensor(output_details[0]['index'])

  # Test the TensorFlow model on random input data.
  tf_results = model(tf.constant(input_data))
  output_data = np.array(tf_results)

  inputs.append(input_data[0][0])
  outputs.append(output_data[0][0])

In [0]:
# Visualize the model
%matplotlib inline

plt.plot(inputs, outputs, 'r')
plt.show()